In [1]:
%pip install "ultralytics<=8.3.40" supervision roboflow
!pip install ray==2.5.1
!pip install -U ultralytics
import ultralytics
ultralytics.checks()

Ultralytics 8.3.163 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 6411.7/8062.4 GB disk)


In [3]:
!yolo task=detect mode=train model=yolo11s.pt data="/kaggle/input/padel-player-dataset/data.yaml" epochs=50 imgsz=640 batch=16

100%|██████████████████████████████████████| 18.4M/18.4M [00:00<00:00, 72.9MB/s]
Ultralytics 8.3.163 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/input/padel-player-dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimi

In [5]:
!pip install deep-sort-realtime opencv-python numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 53.6 MB/s eta 0:00:0000:0100:01


In [9]:
from ultralytics import YOLO

# Load the YOLO model (replace 'yolov8n.pt' with your model path)
model = YOLO('/kaggle/input/padel_model/tensorflow2/default/1/best.pt')

# Get the class names
class_names = model.names

# Print the class names
print("Model class names:")
for idx, name in enumerate(class_names):
    print(f"Class {idx}: {name}")

Model class names:
Class 0: 0
Class 1: 1


In [ ]:
import cv2
import numpy as np
from collections import defaultdict
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

# Initialize YOLO and DeepSORT
model = YOLO("/kaggle/input/padel_model/tensorflow2/default/1/best.pt")
tracker = DeepSort(max_age=5)

# Video setup
cap = cv2.VideoCapture("/kaggle/input/padel-video-test/The best point in padel history-UNBELIVABLE.mp4")
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Output video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output_tracking11.mp4', fourcc, fps, (width, height))

# Tracking variables
player_tracks = defaultdict(list)
KEY_MOMENT_THRESHOLD = 50

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Object Detection
    results = model(frame, classes=1)
    detections = []
    for box in results[0].boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        conf = box.conf[0]
        detections.append(([x1, y1, x2 - x1, y2 - y1], conf, "player"))

    # Tracking
    tracks = tracker.update_tracks(detections, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue
            
        track_id = track.track_id
        ltrb = track.to_ltrb()
        x1, y1, x2, y2 = map(int, ltrb)

        # Store and draw movement
        center = (int((x1 + x2) / 2), int((y1 + y2) / 2))
        player_tracks[track_id].append(center)

        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"Player {track_id}", (x1, y1 - 10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        if len(player_tracks[track_id]) > 1:
            pts = np.array(player_tracks[track_id][-20:], np.int32)
            cv2.polylines(frame, [pts], False, (0, 0, 255), 2)  # Fixed argument order

        # Key moment detection
        if len(player_tracks[track_id]) > 5:
            prev_center = player_tracks[track_id][-5]
            speed = np.sqrt((center[0]-prev_center[0])**2 + (center[1]-prev_center[1])**2)
            if speed > KEY_MOMENT_THRESHOLD:
                cv2.putText(frame, "SMASH!", (x2, y2), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Save frame
    out.write(frame)

# Cleanup
cap.release()
out.release()
print("Processing complete. Output saved as 'output_tracking.mp4'")


0: 384x640 3 Padel-Playerss, 12.4ms
Speed: 1.9ms preprocess, 12.4ms inference, 2.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 Padel-Playerss, 9.2ms
Speed: 1.8ms preprocess, 9.2ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 Padel-Playerss, 9.0ms
Speed: 1.5ms preprocess, 9.0ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Padel-Playerss, 9.1ms
Speed: 1.6ms preprocess, 9.1ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Padel-Playerss, 9.1ms
Speed: 1.7ms preprocess, 9.1ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Padel-Playerss, 9.0ms
Speed: 1.6ms preprocess, 9.0ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Padel-Playerss, 9.2ms
Speed: 1.6ms preprocess, 9.2ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Padel-Playerss, 8.9ms
Speed: 1.6ms preprocess, 8.9ms inferen

In [16]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

# Initialize models
model = YOLO("/kaggle/input/padel_model/tensorflow2/default/1/best.pt")
tracker = DeepSort(max_age=10)

# Video setup
cap = cv2.VideoCapture("/kaggle/input/padel-video-test/The best point in padel history-UNBELIVABLE.mp4")
width, height = int(cap.get(3)), int(cap.get(4))
fps = int(cap.get(5))

# Output writers
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output_tracking.mp4', fourcc, fps, (width, height))

# Tracking data structures
player_data = defaultdict(lambda: {
    'positions': [],
    'speed': [],
    'court_coverage': np.zeros((height, width), dtype=np.uint8)
})

# Visualization parameters
COLORS = [(0,255,0), (0,0,255), (255,0,0), (0,255,255)]  # Different colors per player

frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    frame_count += 1
    frame_copy = frame.copy()
    
    # Player detection
    results = model(frame, classes=1)
    detections = []
    for box in results[0].boxes:
        x1, y1 = map(int, box.xyxy[0][:2])
        w = int(box.xyxy[0][2] - box.xyxy[0][0])
        h = int(box.xyxy[0][3] - box.xyxy[0][1])
        conf = float(box.conf[0])
        detections.append(([x1, y1, w, h], conf, "player"))
    
    # Tracking
    tracks = tracker.update_tracks(detections, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue
            
        track_id = track.track_id
        x1, y1, x2, y2 = map(int, track.to_ltrb())
        center = (int((x1+x2)/2), int((y1+y2)/2))
        
        # Update player data
        player_data[track_id]['positions'].append(center)
        
        # Calculate speed
        if len(player_data[track_id]['positions']) > 5:
            prev_pos = player_data[track_id]['positions'][-5]
            speed = np.sqrt((center[0]-prev_pos[0])**2 + (center[1]-prev_pos[1])**2)
            player_data[track_id]['speed'].append(speed)
            
            # Highlight smashes
            if speed > 50:
                cv2.circle(frame_copy, center, 20, (0,0,255), 3)
                cv2.putText(frame_copy, f"SMASH! {speed:.0f}px", (x2+10,y2), 
                          cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
        
        # Draw movement path
        if len(player_data[track_id]['positions']) > 1:
            pts = np.array(player_data[track_id]['positions'], np.int32)
            cv2.polylines(frame_copy, [pts], False, COLORS[track_id%len(COLORS)], 2)
            
            # Draw direction arrows
            if len(player_data[track_id]['positions']) > 50:
                for i in range(50, len(player_data[track_id]['positions']), 50):
                    cv2.arrowedLine(frame_copy, 
                                  player_data[track_id]['positions'][i-1],
                                  player_data[track_id]['positions'][i],
                                  COLORS[track_id%len(COLORS)], 2, tipLength=0.5)
        
        # Update court coverage
        cv2.circle(player_data[track_id]['court_coverage'], center, 15, 255, -1)
        
        # Current position marker - FIXED STRING FORMATTING
        cv2.circle(frame_copy, center, 8, (255,255,255), -1)
        cv2.putText(frame_copy, f"Player {track_id}", (x1,y1-15),  # Changed from f"P{track_id}"
                   cv2.FONT_HERSHEY_SIMPLEX, 0.8, COLORS[track_id%len(COLORS)], 2)

    # Add frame counter
    cv2.putText(frame_copy, f"Frame: {frame_count}", (20,40), 
               cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
    
    # Save frame
    out.write(frame_copy)

# Generate visualizations
plt.figure(figsize=(20,10))
for pid, data in player_data.items():
    if len(data['positions']) > 50:
        x, y = zip(*data['positions'])
        plt.plot(x, y, label=f'Player {pid}', linewidth=2)
        plt.scatter(x[-1], y[-1], s=100, marker='o')
        
plt.title('Player Movement Paths', fontsize=16)
plt.xlabel('Court X Position', fontsize=14)
plt.ylabel('Court Y Position', fontsize=14)
plt.gca().invert_yaxis()
plt.legend(fontsize=12)
plt.grid(True)
plt.savefig('movement_paths.png', dpi=300, bbox_inches='tight')

plt.figure(figsize=(20,15))
for i, (pid, data) in enumerate(player_data.items(), 1):
    if len(data['positions']) > 50:
        plt.subplot(2, 2, i)
        plt.imshow(data['court_coverage'], cmap='hot', alpha=0.7)
        plt.title(f'Player {pid} Coverage', fontsize=14)
        plt.colorbar()
plt.savefig('coverage_heatmaps.png', dpi=300, bbox_inches='tight')

# Cleanup
cap.release()
out.release()
print("""
Processing complete:
1. output_tracking.mp4 - Video with tracking visualization
2. movement_paths.png - Player movement paths
3. coverage_heatmaps.png - Court coverage heatmaps
""")


0: 384x640 3 Padel-Playerss, 12.7ms
Speed: 1.9ms preprocess, 12.7ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 Padel-Playerss, 8.9ms
Speed: 1.5ms preprocess, 8.9ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 Padel-Playerss, 9.0ms
Speed: 1.5ms preprocess, 9.0ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)


TypeError: not all arguments converted during string formatting

with g

In [14]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

# Initialize YOLO and DeepSORT
model = YOLO("/kaggle/input/padel_model/tensorflow2/default/1/best.pt")
tracker = DeepSort(max_age=5)

# Video setup
cap = cv2.VideoCapture("/kaggle/input/padel-video-test/The best point in padel history-UNBELIVABLE.mp4")
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Output video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output_tracking22.mp4', fourcc, fps, (width, height))

# Tracking and graph data
player_tracks = defaultdict(list)
graph_data = defaultdict(lambda: {'x': [], 'y': [], 'speed': []})
KEY_MOMENT_THRESHOLD = 50
frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    frame_count += 1
    
    # Object Detection
    results = model(frame, classes=1)
    detections = []
    for box in results[0].boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        conf = box.conf[0]
        detections.append(([x1, y1, x2-x1, y2-y1], conf, "player"))

    # Tracking
    tracks = tracker.update_tracks(detections, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue
            
        track_id = track.track_id
        ltrb = track.to_ltrb()
        x1, y1, x2, y2 = map(int, ltrb)
        center = (int((x1+x2)/2), int((y1+y2)/2))
        
        # Store movement data
        player_tracks[track_id].append(center)
        graph_data[track_id]['x'].append(frame_count)
        graph_data[track_id]['y'].append(center[0])  # Using x-position for graph
        
        # Calculate speed
        if len(player_tracks[track_id]) > 5:
            prev_center = player_tracks[track_id][-5]
            speed = np.sqrt((center[0]-prev_center[0])**2 + (center[1]-prev_center[1])**2)
            graph_data[track_id]['speed'].append(speed)
            
            # Key moment detection
            if speed > KEY_MOMENT_THRESHOLD:
                cv2.putText(frame, f"SMASH! {speed:.1f}px/f", (x2, y2), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Draw bounding box and ID
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"P{track_id}", (x1, y1-10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
        # Draw movement path
        if len(player_tracks[track_id]) > 1:
            pts = np.array(player_tracks[track_id][-30:], np.int32)
            cv2.polylines(frame, [pts], False, (0, 0, 255), 2)

    # Save frame
    out.write(frame)

# Generate and save movement graphs
plt.figure(figsize=(15, 8))
for player_id, data in graph_data.items():
    if len(data['x']) > 10:  # Only plot players with sufficient data
        plt.plot(data['x'], data['y'], label=f'Player {player_id}')
        
plt.title('Player Movement Patterns')
plt.xlabel('Frame Number')
plt.ylabel('X-Position (Court Position)')
plt.legend()
plt.grid(True)
plt.savefig('player_movement_graph.png')
plt.close()

# Generate speed analysis graph
plt.figure(figsize=(15, 8))
for player_id, data in graph_data.items():
    if len(data['speed']) > 5:
        plt.plot(data['x'][5:], data['speed'], label=f'Player {player_id}')
        
plt.title('Player Speed Analysis')
plt.xlabel('Frame Number')
plt.ylabel('Speed (pixels/frame)')
plt.axhline(y=KEY_MOMENT_THRESHOLD, color='r', linestyle='--', label='Smash Threshold')
plt.legend()
plt.grid(True)
plt.savefig('player_speed_graph.png')
plt.close()

# Cleanup
cap.release()
out.release()
print("""
Processing complete!
- Tracking video saved as 'output_tracking.mp4'
- Movement graph saved as 'player_movement_graph.png'
- Speed analysis saved as 'player_speed_graph.png'
""")


0: 384x640 3 Padel-Playerss, 10.1ms
Speed: 1.8ms preprocess, 10.1ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 Padel-Playerss, 10.5ms
Speed: 1.8ms preprocess, 10.5ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 Padel-Playerss, 9.3ms
Speed: 1.7ms preprocess, 9.3ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Padel-Playerss, 9.3ms
Speed: 1.6ms preprocess, 9.3ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Padel-Playerss, 9.6ms
Speed: 1.6ms preprocess, 9.6ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Padel-Playerss, 9.2ms
Speed: 1.7ms preprocess, 9.2ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Padel-Playerss, 9.3ms
Speed: 1.7ms preprocess, 9.3ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 Padel-Playerss, 9.2ms
Speed: 1.8ms preprocess, 9.2ms infer

In [ ]:
import shutil

# Path to the folder you want to zip
folder_to_zip = "/kaggle/working/runs/detect/train"

# Output ZIP file name
output_zip_path = "/kaggle/working/predict"

# Compress the folder
shutil.make_archive(output_zip_path.replace('.zip', ''), 'zip', folder_to_zip)

print(f"Folder '{folder_to_zip}' successfully compressed into '{output_zip_path}'")
